In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

%matplotlib inline

mpl.rcParams['figure.figsize'] = (16, 10)
pd.set_option('display.max_rows', 500)
import plotly
import plotly.graph_objects as go

In [2]:
plotly.__version__

'4.6.0'

In [3]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  

df_analyse.sort_values('date',ascending=True).tail()

,date,Italy,US,Spain,Germany,Nepal,India
232,2020-09-10,283180,6396100,554143,258149,50465,4562414
233,2020-09-11,284796,6443652,566326,259735,51919,4659984
234,2020-09-12,286297,6485123,566326,260817,53120,4754356
235,2020-09-13,287753,6519573,566326,261737,54159,4846427
236,2020-09-14,288761,6553652,593730,263222,55329,4846427


In [4]:
def quick_plot(x_in, df_input,y_scale='log',slider=False):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
    
        
        Returns:
        ----------
        
    """
    fig = go.Figure()

    for each in df_input.columns:
        fig.add_trace(go.Scatter(
                        x=x_in,
                        y=df_input[each],
                        name=each,
                        opacity=0.8))
    
    fig.update_layout(autosize=True,
        width=1024,
        height=768,
        font=dict(
            family="PT Sans, monospace",
            size=18,
            color="#7f7f7f"
            )
        )
    fig.update_yaxes(type=y_scale),
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show(renderer="firefox")
#     fig.show()

In [5]:
quick_plot(df_analyse.date,
           df_analyse.iloc[:,1:],
           y_scale='linear',
           slider=True)

In [6]:
threshold=100

compare_list=[]
for pos,country in enumerate(df_analyse.columns[1:]):
    compare_list.append(np.array(df_analyse[country][df_analyse[country]>threshold]))

pd_sync_timelines=pd.DataFrame(compare_list,index=df_analyse.columns[1:]).T

pd_sync_timelines['date']=np.arange(pd_sync_timelines.shape[0])

pd_sync_timelines.head()



,Italy,US,Spain,Germany,Nepal,India,date
0,155.0,103.0,120.0,130.0,101.0,102.0,0
1,229.0,172.0,165.0,159.0,102.0,113.0,1
2,322.0,215.0,222.0,196.0,110.0,119.0,2
3,453.0,337.0,259.0,262.0,110.0,142.0,3
4,655.0,450.0,400.0,482.0,134.0,156.0,4


In [7]:
quick_plot(pd_sync_timelines.date,
           pd_sync_timelines.iloc[:,:-1],
           y_scale='log',
           slider=True)

## Doubling rate

In [8]:
def doubling_rate(N_0,t,T_d):
    return N_0*np.power(2,t/T_d)

In [9]:
max_days=180

norm_slopes={
    #'doubling every day':doubling_rate(100,np.arange(10),1),
    'doubling every two days':doubling_rate(100,np.arange(max_days),2),
    'doubling every 4 days':doubling_rate(100,np.arange(max_days),4),
    'doubling every 10 days':doubling_rate(100,np.arange(max_days),10),
}

In [10]:
pd_sync_timelines_w_slope=pd.concat([pd.DataFrame(norm_slopes),pd_sync_timelines], axis=1)

In [11]:
quick_plot(pd_sync_timelines_w_slope.date,
           pd_sync_timelines_w_slope.iloc[:,0:5],
           y_scale='log',
           slider=True)

In [12]:
pd_sync_timelines_w_slope.to_csv('../data/processed/COVID_small_sync_timeline_table.csv',sep=';',index=False)

## Linear regression

In [13]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=False)


In [14]:
l_vec=len(df_analyse['Nepal'])
X=np.arange(l_vec-5).reshape(-1, 1)
y=np.log(np.array(df_analyse['Nepal'][5:]))

# do a simple regression on all data

reg.fit(X,y)

X_hat=np.arange(l_vec).reshape(-1, 1)
Y_hat=reg.predict(X_hat)

LR_inspect=df_analyse[['date','Nepal']].copy()

LR_inspect['prediction']=np.exp(Y_hat)

In [15]:
quick_plot(LR_inspect.date,
           LR_inspect.iloc[:,1:],
           y_scale='log',
           slider=True)

## Piecewise Linear Regression Doubling rate

In [16]:
from sklearn import linear_model
from scipy import signal

reg = linear_model.LinearRegression(fit_intercept=True)

df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  

country_list=df_analyse.columns[1:]




In [17]:
country_list

Index(['Italy', 'US', 'Spain', 'Germany', 'Nepal', 'India'], dtype='object')

In [18]:
## filter data

for each in country_list:
    df_analyse[each+'_filter']=signal.savgol_filter(df_analyse[each],
                           5, # window size used for filtering
                           1) # order of fitted polynomial

In [19]:
# filter_cols=['Italy_filter','US_filter', 'Spain_filter', 'Germany_filter']

filter_cols=df_analyse.columns[len(country_list)+1:]  #after date and country list, remaining are filters


In [20]:
filter_cols

Index(['Italy_filter', 'US_filter', 'Spain_filter', 'Germany_filter',
       'Nepal_filter', 'India_filter'],
      dtype='object')

In [21]:
df_analyse.head()

,date,Italy,US,Spain,Germany,Nepal,India,Italy_filter,US_filter,Spain_filter,Germany_filter,Nepal_filter,India_filter
0,2020-01-22,0,1,0,0,0,0,0.0,0.4,0.0,0.0,-0.2,0.0
1,2020-01-23,0,1,0,0,0,0,0.0,1.3,0.0,0.0,0.1,0.0
2,2020-01-24,0,2,0,0,0,0,0.0,2.2,0.0,0.0,0.4,0.0
3,2020-01-25,0,2,0,0,1,0,0.0,3.0,0.0,0.2,0.6,0.0
4,2020-01-26,0,5,0,0,1,0,0.0,3.8,0.0,1.0,0.8,0.0


In [22]:
df_analyse[filter_cols].columns

Index(['Italy_filter', 'US_filter', 'Spain_filter', 'Germany_filter',
       'Nepal_filter', 'India_filter'],
      dtype='object')

In [23]:
df_analyse[filter_cols].iloc[3:5,3:5]

,Germany_filter,Nepal_filter
3,0.2,0.6
4,1.0,0.8


In [24]:
start_pos=5
quick_plot(df_analyse.date[start_pos:],
           df_analyse[filter_cols].iloc[start_pos:,:], #['US','US_filter']
           y_scale='log',
           slider=True)

In [25]:
def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    return intercept/slope


In [26]:
def doubling_time(in_array):
    ''' Use a classical doubling time formular, 
     see https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [27]:
# calculate slope of regression of last x days
# use always a limited number of days to approximate the triangle, attention exponential base assumption
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(country_list):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [28]:
# run on all filtered data
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(filter_cols):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [29]:
# cross check the matematical 
df_analyse['Germany_DR_math']=df_analyse['Germany'].rolling(
                                window=days_back,
                                min_periods=days_back).apply(doubling_time, raw=False)

In [30]:
# run on all filtered data
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(filter_cols):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [34]:
df_analyse.head()

,date,Italy,US,Spain,Germany,Nepal,India,Italy_filter,US_filter,Spain_filter,...,Germany_DR,Nepal_DR,India_DR,Italy_filter_DR,US_filter_DR,Spain_filter_DR,Germany_filter_DR,Nepal_filter_DR,India_filter_DR,Germany_DR_math
0,2020-01-22,0,1,0,0,0,0,0.0,0.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,0,1,0,0,0,0,0.0,1.3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-24,0,2,0,0,0,0,0.0,2.2,0.0,...,NaN,NaN,NaN,NaN,1.444444,NaN,NaN,0.333333,NaN,NaN
3,2020-01-25,0,2,0,0,1,0,0.0,3.0,0.0,...,NaN,0.666667,NaN,NaN,2.549020,NaN,0.666667,1.466667,NaN,NaN
4,2020-01-26,0,5,0,0,1,0,0.0,3.8,0.0,...,NaN,1.333333,NaN,NaN,3.750000,NaN,0.800000,3.000000,NaN,NaN


In [32]:
start_pos=70
quick_plot(df_analyse.date[start_pos:],
           df_analyse.iloc[start_pos:,[13,14,15,16,17,18]], #
           y_scale='linear',
           slider=True)

In [33]:
start_pos=70
quick_plot(df_analyse.date[start_pos:],
           df_analyse.iloc[start_pos:,[19,20,21,22,23,24]], #
           y_scale='linear',
           slider=True)